In [1]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bigdatalab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. 데이터 다운로드

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x2af9a047760>)

# 2. 데이터 전처리하기

In [4]:
# 실질적 데이터(영어 문장)은 <content> 와 </content>사이에 있다
# 데이터 안에서 Laughter이나 Applause와 같은 배경음을 나타내는 단어를 제거해야 한다

targetXML = open("ted_en-20160408.xml", "r", encoding = "UTF8")
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath("//content/text()"))
# xml파일에서 <content>와 </content> 사이의 내용만 가져온다

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거

sent_text = sent_tokenize(content_text)
# 문장 토크화를 수행

normalized_text = []
for string in sent_text:
    tokens = tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 단어 토큰화를 수행

In [5]:
print("총 sample의 개수 :", len(result))

총 sample의 개수 : 273424


In [6]:
print(result[0:3])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']]


# 3. Word2Vec 훈련시키기

In [8]:
from gensim.models import Word2Vec

In [10]:
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

# size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.

In [11]:
# "man"과 가장 유사한 단어들을 출력
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8474211692810059), ('guy', 0.8087135553359985), ('lady', 0.7892827391624451), ('boy', 0.772880494594574), ('gentleman', 0.7623029947280884), ('girl', 0.7422564029693604), ('kid', 0.7335081696510315), ('soldier', 0.7206511497497559), ('friend', 0.666482150554657), ('poet', 0.6654611825942993)]


# 4. Word2Vec 모델 저장하고 로드하기

In [20]:
from gensim.models import KeyedVectors

In [23]:
model.wv.save_word2vec_format("./eng_w2v") # 모델 저장
load_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [25]:
model_result = load_model.most_similar("man")
print(model_result)

[('woman', 0.8474211692810059), ('guy', 0.8087135553359985), ('lady', 0.7892827391624451), ('boy', 0.772880494594574), ('gentleman', 0.7623029947280884), ('girl', 0.7422564029693604), ('kid', 0.7335081696510315), ('soldier', 0.7206511497497559), ('friend', 0.666482150554657), ('poet', 0.6654611825942993)]


In [30]:
print(load_model.get_vector("man"))

[ 1.0296485  -0.19767332 -0.54695266 -1.1040423   2.765395   -1.1815919
 -1.6822832  -0.21522988  0.5867903  -1.0269867  -0.658276    0.66991025
  1.4628726   2.1555598   1.4616125  -0.3365607   1.2890157   0.06718137
  0.69378597 -0.5847847  -0.792276   -0.1956609   0.2115657   0.6519621
  2.5184119   1.3139853  -0.4153869  -1.9899923  -0.5464799  -0.86693084
 -0.07208136 -1.7868081   0.7531264  -0.01804415 -0.823373   -0.5274344
  2.1360114   0.83172405 -0.9740203   0.04703219 -0.27589673 -1.7699386
 -0.02038041 -0.17686105 -0.11595158  0.00326262  0.26981306 -0.32457045
 -0.47317737  0.99758554  0.7315955   0.4134463  -1.2931443  -0.01127165
 -1.1691226   1.0294603   0.38621402 -0.113755    1.1200052   0.26051483
  1.3926753  -0.25442615 -1.1459651  -0.46785596  0.10317799  1.4614317
  1.3510655   1.0352789   0.5426012   0.8065396   0.83292276  1.1050802
  1.3798331  -1.1012571  -0.47983423  0.7246187   0.4981369  -0.10618542
 -0.14805187  0.15769252 -0.57696235  1.9021116   1.12173

In [31]:
print(load_model.get_vector("man").shape)

(100,)
